In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- Step 1: setup (imports + file paths) ---
import os

BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/CW_App_BI"

#Curated gene sets from Saccharomyces Genome Database(SGD) by pathway
#Clean Files containg only the gene names (
#Downloaded annotation files were cleaned using sed and awk commands during differential expression analysis to check whether the results have significant genes in expected pathways)
#The clean files are re-purposed here
GENESET_FILES = {
    "amino_acid": os.path.join(BASE_PATH, "gene_sets", "genes_amino_acid_biosynthesis_clean.txt"),
    "glycolysis": os.path.join(BASE_PATH, "gene_sets", "genes_glycolysis_clean.txt"),

}


#Cufflinks output files with expression level values(FPKM)
FPKM_PATHS = {
    "gln3_0_rep1":  os.path.join(BASE_PATH, "cufflinks", "gln3-0_rep1_genes.fpkm_tracking"),
    "gln3_0_rep2":   os.path.join(BASE_PATH, "cufflinks", "gln3-0_rep2_genes.fpkm_tracking"),
    "gln3_iso_rep1": os.path.join(BASE_PATH, "cufflinks", "gln3-i-BuOH_rep1_genes.fpkm_tracking"),
    "gln3_iso_rep2": os.path.join(BASE_PATH, "cufflinks", "gln3-i-BuOH_rep2_genes.fpkm_tracking"),
    "wt_iso_rep1":  os.path.join(BASE_PATH, "cufflinks", "WT-i-BuOH_rep1_genes.fpkm_tracking"),
    "wt_iso_rep2":  os.path.join(BASE_PATH, "cufflinks", "WT-i-BuOH_rep2_genes.fpkm_tracking")

}

#Cuffdiff was run on: WT0,gln3_iso,WT_iso,gln3_0 together to generate a combined gene_exp.diff (renamed to combined_gene_exp.diff)
COMBINED_CUFFDIFF_PATH = os.path.join(BASE_PATH, "combined_gene_exp.diff")
combined_diff = pd.read_csv(COMBINED_CUFFDIFF_PATH, sep="\t")

print("Unique sample_1:", combined_diff["sample_1"].unique())
print("Unique sample_2:", combined_diff["sample_2"].unique())


HEATMAP_CONDITIONS = ["gln3_0", "gln3_iso", "wt_iso"]

In [ ]:
# --- Step 2: load cufflinks FPKM files + build 3-condition matrix (replicate-averaged) ---

def load_fpkm_tracking(path):

    #Loads a Cufflinks genes.fpkm_tracking file and returns a tidy dataframe with:
    #gene_id, gene_short_name, FPKM

    df = pd.read_csv(path, sep="\t")
    needed = ["gene_id", "gene_short_name", "FPKM"]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in {path}: {missing}\nFound: {list(df.columns)}")

    # Keep only what is needed ("gene_id", "gene_short_name", "FPKM")
    df = df[needed].copy()

    # Ensure all FPKM values are numeric
    df["FPKM"] = pd.to_numeric(df["FPKM"], errors="coerce").fillna(0.0)

    return df


# Load all 6 files
gln3_0_rep1   = load_fpkm_tracking(FPKM_PATHS["gln3_0_rep1"])
gln3_0_rep2   = load_fpkm_tracking(FPKM_PATHS["gln3_0_rep2"])
gln3_iso_rep1 = load_fpkm_tracking(FPKM_PATHS["gln3_iso_rep1"])
gln3_iso_rep2 = load_fpkm_tracking(FPKM_PATHS["gln3_iso_rep2"])
wt_iso_rep1   = load_fpkm_tracking(FPKM_PATHS["wt_iso_rep1"])
wt_iso_rep2   = load_fpkm_tracking(FPKM_PATHS["wt_iso_rep2"])


# Calculate average of replicates(1&2) by gene_id
def mean_reps(df1, df2, colname):
    m = df1.merge(df2, on="gene_id", suffixes=("_rep1", "_rep2"), how="inner")
    # keep representative short name (rep1)
    m["gene_short_name"] = m["gene_short_name_rep1"]
    m[colname] = (m["FPKM_rep1"] + m["FPKM_rep2"]) / 2.0
    return m[["gene_id", "gene_short_name", colname]]

gln3_0_mean   = mean_reps(gln3_0_rep1,   gln3_0_rep2,   "gln3_0_FPKM")
gln3_iso_mean = mean_reps(gln3_iso_rep1, gln3_iso_rep2, "gln3_iso_FPKM")
wt_iso_mean   = mean_reps(wt_iso_rep1,   wt_iso_rep2,   "wt_iso_FPKM")


# Merge all 3 means into one expression table (gene_id is the join key)

expr_df = (
    gln3_0_mean[["gene_id", "gene_short_name", "gln3_0_FPKM"]]
      .merge(gln3_iso_mean[["gene_id", "gln3_iso_FPKM"]], on="gene_id", how="inner")
      .merge(wt_iso_mean[["gene_id", "wt_iso_FPKM"]],     on="gene_id", how="inner")
)


# Add log2(FPKM+1) columns (the paper uses log2FPKM; +1 avoids log2(0))
for col in ["gln3_0_FPKM", "gln3_iso_FPKM", "wt_iso_FPKM"]:
    expr_df["log2_" + col] = np.log2(expr_df[col] + 1.0)

display(expr_df.head())
print("Genes in merged 3-condition table:", len(expr_df))


In [ ]:
expr_df.columns.tolist()

In [ ]:
# --- Step 3: load SGD-derived pathway gene sets ---

def load_gene_set(path):
    with open(path) as f:
        genes = [line.strip() for line in f if line.strip()]
    return set(genes)   #removes duplicates


# Using SGD-derived files (cleaned)
genes_amino_acid = load_gene_set(
    GENESET_FILES["amino_acid"]
)

genes_glycolysis = load_gene_set(
    GENESET_FILES["glycolysis"]
)

print("Amino acid biosynthesis genes:", len(genes_amino_acid))
print("Glycolysis genes:", len(genes_glycolysis))

In [ ]:
# --- Step 4: subseting expression matrix to pathway genes separating amino acid and glycolysis ---

expr_aa_df = expr_df[expr_df["gene_short_name"].isin(genes_amino_acid)].copy()
expr_gly_df = expr_df[expr_df["gene_short_name"].isin(genes_glycolysis)].copy()

print("AA genes in expr_df:", expr_aa_df["gene_id"].nunique())
print("Glycolysis genes in expr_df:", expr_gly_df["gene_id"].nunique())

In [ ]:
# --- Step 5 : Creating log2 colnames list---
log_cols = [
    "log2_gln3_0_FPKM",
    "log2_gln3_iso_FPKM",
    "log2_wt_iso_FPKM"
]

In [ ]:
# --- Step 6: compute z scores ---
#This function rescales each gene’s expression values so we can see which conditions are high or low
#relative to the gene’s own average
def compute_row_zscores(df_param, log_cols):
    z = df_param[log_cols].sub(df_param[log_cols].mean(axis=1), axis=0)
    #print(z.head())
    #dividing by standard deviation will crash if sd=0, so replace 0 by nan
    z = z.div(df_param[log_cols].std(axis=1).replace(0, np.nan), axis=0)
    return z

In [ ]:
# --- Step 7: Call compute_row_zscores for amino acid and glycolysis pathways ---
aa_z = compute_row_zscores(expr_aa_df, log_cols)
print(aa_z.head)
gly_z = compute_row_zscores(expr_gly_df, log_cols)
#print(gly_z.head)

In [ ]:
# --- Step 8: Add gene names back for plotting and Drop genes with NaN Z scores if any from the matrices ---
#Creating row labels for the two matrices
aa_z.index = expr_aa_df["gene_short_name"].values
gly_z.index = expr_gly_df["gene_short_name"].values


aa_z_plot  = aa_z.dropna()
gly_z_plot = gly_z.dropna()

print("AA genes plotted:", aa_z_plot.shape[0])
print("Glycolysis genes plotted:", gly_z_plot.shape[0])

In [ ]:
print(aa_z.index)

In [ ]:
print(gly_z.index)

In [ ]:
# ---Step 9: get p-values from cuffdiff results to add * to heatmap ---

def pval_map(df):
    df = df[df["status"] == "OK"].copy()
    return df.groupby("gene")["p_value"].min().to_dict()


def pval_map_pair(diff_df, a, b):
    x = diff_df[(diff_df["status"]=="OK") &
                (((diff_df["sample_1"]==a) & (diff_df["sample_2"]==b)) |
                 ((diff_df["sample_1"]==b) & (diff_df["sample_2"]==a)))]
    return x.groupby("gene")["p_value"].min().to_dict()


p_gln3_0_vs_wt0    = pval_map_pair(combined_diff, "gln3_0",   "WT_0")
p_gln3iso_vs_wt0   = pval_map_pair(combined_diff, "gln3_iso", "WT_0")
p_wtiso_vs_wt0     = pval_map_pair(combined_diff, "WT_iso",   "WT_0")

print("Dict size:", len(p_gln3_0_vs_wt0))
print("Dict size:", len(p_gln3iso_vs_wt0))
print("Dict size:", len(p_wtiso_vs_wt0))


In [ ]:
print(p_gln3_0_vs_wt0)
print(combined_diff.head())

In [ ]:
# --- Step 10: Setting conditions to add stars to amino acid heatmap cells ---

HEAT_COLS_AA = ["log2_gln3_0_FPKM", "log2_gln3_iso_FPKM", "log2_wt_iso_FPKM"]

stars = pd.DataFrame("", index=aa_z_plot.index, columns=HEAT_COLS_AA)

# gln3Δ (no iBuOH) vs WT_0
#Loop on genes in aa_z_plot.index and match in p_gln3_0_vs_wt0 dict to filter those with p-value <0.05
stars["log2_gln3_0_FPKM"] = [
    "*" if (p_gln3_0_vs_wt0.get(gene, 1.0) < 0.05) else "" for gene in aa_z_plot.index  #if gene is not found in dict p_gln3_0_vs_wt0.get, return 1.0 which is not significant so gets excluded
]

# gln3Δ (+ iBuOH) vs WT0
stars["log2_gln3_iso_FPKM"] = [
    "*" if (p_gln3iso_vs_wt0.get(gene, 1.0) < 0.05) else "" for gene in aa_z_plot.index
]


# WT (iBuOH) vs WT_0
stars["log2_wt_iso_FPKM"] = [
    "*" if (p_wtiso_vs_wt0.get(gene, 1.0) < 0.05) else "" for gene in aa_z_plot.index
]


In [ ]:
# --- Step 11: choose AA heatmap genes using paper DEG(differentially expressed genes) criteria (WT_iso vs WT_0) and subset z-scores & stars ---

#Keep only tested genes and compute paper DEG flag
combined_diff = combined_diff[combined_diff["status"] == "OK"].copy() #Removes NOTEST rows
combined_diff["deg_paper"] = (combined_diff["p_value"] < 0.05) & (combined_diff["log2(fold_change)"].abs() > 1)

#print(combined_diff.head())

# Restrict to WT_iso vs WT_0 comparison
wt_iso_vs_wt0 = combined_diff[
    (combined_diff["sample_1"] == "WT_iso") &
    (combined_diff["sample_2"] == "WT_0")
].copy()


# Restrict to amino-acid pathway genes (by gene symbol)

wt_aa_deg = wt_iso_vs_wt0[
    wt_iso_vs_wt0["gene"].isin(genes_amino_acid) &
    wt_iso_vs_wt0["deg_paper"]
].copy()

print("AA DEGs in WT_iso vs WT_0:", wt_aa_deg["gene"].nunique())

# Limit how many genes to show on heatmap (rank by |log2FC|)
N = 10
top_aa_genes = (
    wt_aa_deg.sort_values(by="log2(fold_change)", key=lambda s: s.abs(), ascending=False)
             ["gene"]
             .drop_duplicates()
             .head(N)
             .tolist()
)

print(f"Showing top {len(top_aa_genes)} AA genes by |log2FC| for heatmap.")

# Subset z-scores + stars to these genes (these indices are gene_short_name)
aa_subset = aa_z_plot.loc[aa_z_plot.index.intersection(top_aa_genes)]
stars_subset = stars.loc[stars.index.intersection(aa_subset.index)]

print("aa_subset shape:", aa_subset.shape)
print("stars_subset shape:", stars_subset.shape)
print(list(wt_aa_deg['gene']))
print(top_aa_genes)

In [ ]:
#Checking how many amino acid genes match to the amino acid genes in the paper's heatmap

aa_genes_paper = ['ILV5', 'ILV2', 'ILV3', 'BAT1', 'LEU1', 'BAT2', 'LEU4', 'LEU9', 'ARO3', 'ARO4', 'TRP3', 'ARO1', 'ARO8', 'TRP2', 'ARO9', 'HOM2', 'MET14', 'MET1', 'MET10', 'MET2', 'HOM3', 'MET6', 'MET16', 'MET3', 'MET5', 'ARG1', 'ARG3', 'ARG4', 'ARG8', 'ARG5,6', 'ARG7', 'GLT1', 'GDH1', 'GDH3', 'IDP1', 'GLN1', 'HIS4', 'HIS5', 'HIS1', 'HIS7', 'SER2', 'SER1', 'SER33', 'SER3', 'LYS1', 'LYS20', 'LYS9', 'CYS3', 'CYS4', 'AAT1', 'ASN1', 'ALT1', 'CDC19', 'ENO2', 'ENO1', 'FBA1', 'TDH1', 'PGK1', 'TPI1', 'TDH3', 'GPM1', 'TDH2', 'PYK2']
aa_genes_rep_study = wt_aa_deg['gene']
def common_elements_preserve_order(list1, list2):
    set2 = set(list2)
    return [x for x in list1 if x in set2]


common_aa = common_elements_preserve_order(aa_genes_rep_study, aa_genes_paper)
print(len(common_aa))


print(len(aa_genes_paper))
print(len(aa_genes_rep_study))

In [ ]:
# --- Step 12 Creating a custom colour map to reflect colours like in the paper ---

from matplotlib.colors import LinearSegmentedColormap

paper_cmap = LinearSegmentedColormap.from_list(
    "paper_blue_black_yellow",
    ["#00AAFF", "#000000", "#FFF800"],  # blue->black->yellow
    N=256
)


In [ ]:
# --- Step 13: Heatmap plot function ---
def plot_pathway_heatmap(z_df, title, figsize=(3, 8)):
    g = sns.clustermap(
        z_df,
        col_cluster=False,     # keep columns in fixed order
        row_cluster=True,      # hierarchical clustering of genes
        cmap=paper_cmap,
        center=0,              # z-scores centered at 0
        figsize=figsize,
        linewidths=0.0,
        dendrogram_ratio=(0.15, 0.05)
    )

    # Colorbar label
    g.cax.set_ylabel("Row Z-score", rotation=270, labelpad=15)

    # Get the exact row/col order used in the plotted heatmap
    row_order = g.dendrogram_row.reordered_ind        # indices of rows in plotted order
    col_order = list(range(z_df.shape[1]))

    # Reindex stars matrix to match the plotted heatmap order
    stars_subset = stars.loc[z_df.index, z_df.columns]   # same genes/cols as the plotted data
    stars_subset = stars_subset.iloc[row_order, col_order]

    # Draw the stars at the center of each cell
    ax = g.ax_heatmap
    for i in range(stars_subset.shape[0]):
        for j in range(stars_subset.shape[1]):
            s = stars_subset.iat[i, j]
            if s:  # only draw if "*" (or non-empty)
                ax.text(j + 0.5, i + 0.5, s,
                        ha="center", va="center",
                        color="red", fontsize=9, fontweight="bold")


    # Move column labels to the TOP
    g.ax_heatmap.xaxis.tick_top()
    g.ax_heatmap.xaxis.set_label_position('top')

    # Rotate labels (Prevents label overlap)
    g.ax_heatmap.set_xticklabels(
        g.ax_heatmap.get_xticklabels(),
        rotation=90
    )

    # Move colorbar to the right: [left, bottom, width, height]
    g.cax.set_position([1.2, 0.6, 0.1, 0.2])

    # Make cells square
    g.ax_heatmap.set_aspect("equal")

    plt.suptitle(title, y=1.2)
    plt.show()


In [ ]:
# --- Step 14: Plot heatmap for Amino Acid biosynthesis genes ---

plot_pathway_heatmap(
    aa_subset,
    title="Amino acid biosynthesis genes (row Z-scores)"
)

In [ ]:
# --- Step 15: Setting conditions to add stars to glycolysis heatmap cells ---

HEAT_COLS_GLY = ["log2_gln3_0_FPKM", "log2_gln3_iso_FPKM", "log2_wt_iso_FPKM"]

stars = pd.DataFrame("", index=gly_z_plot.index, columns=HEAT_COLS_GLY)

# gln3Δ (no iBuOH) vs WT_0
stars["log2_gln3_0_FPKM"] = [
    "*" if (p_gln3_0_vs_wt0.get(gene, 1.0) < 0.05) else "" for gene in gly_z_plot.index
]

# gln3Δ (+ iBuOH) vs WT0
stars["log2_gln3_iso_FPKM"] = [
    "*" if (p_gln3iso_vs_wt0.get(gene, 1.0) < 0.05) else "" for gene in gly_z_plot.index
]

# WT (iBuOH) vs WT_0
stars["log2_wt_iso_FPKM"] = [
    "*" if (p_wtiso_vs_wt0.get(gene, 1.0) < 0.05) else "" for gene in gly_z_plot.index
]

In [ ]:
# --- Step 16 (glycolysis): choose glycolysis heatmap genes using paper DEG criteria (WT_iso vs WT_0) ---

# Restrict to glycolysis pathway genes
wt_gly_deg = wt_iso_vs_wt0[
    wt_iso_vs_wt0["gene"].isin(genes_glycolysis) &
    wt_iso_vs_wt0["deg_paper"]
].copy()

print("Glycolysis DEGs in WT_iso vs WT_0 (paper criteria):", wt_gly_deg["gene"].nunique())

# Pick top genes by |log2FC|
N_GLY = 10
top_gly_genes = (
    wt_gly_deg.sort_values(by="log2(fold_change)", key=lambda s: s.abs(), ascending=False)
              ["gene"]
              .drop_duplicates()
              .head(N_GLY)
              .tolist()
)

print(f"Showing top {len(top_gly_genes)} glycolysis genes by |log2FC| for heatmap.")

# Subset z-scores + stars (indices are gene_short_name)
gly_subset = gly_z_plot.loc[gly_z_plot.index.intersection(top_gly_genes)]
gly_stars_subset = stars.loc[stars.index.intersection(gly_subset.index)]

print("gly_subset shape:", gly_subset.shape)
print("gly_stars_subset shape:", gly_stars_subset.shape)
print(top_gly_genes)

In [ ]:
# --- Step 17: Plot heatmap for glycolysis genes ---
plot_pathway_heatmap(
    gly_subset,
    title="Glycolysis genes (row Z-scores)"
)
